In [24]:
from dcicutils import ff_utils
from functions.notebook_functions import *

# get key from file
my_key = get_key('koray_data')

# all usable env names
all_envs = ['data', 'staging', 'fourfront-webdev', 'fourfront-mastertest', 'fourfront-hotseat']
# get admin key from s3
#my_key = ff_utils.get_authentication_with_server({}, ff_env = 'data')

In [25]:
q = '/search/?type=Workflow'
all_wf = ff_utils.search_metadata(q, key=my_key)

In [26]:
print(len(all_wf))

45


In [27]:
allowed_formats =  ["fastq",
                    "bam",
                    "bai",
                    "pairs",
                    "pairsam",
                    "pairs_px2",
                    "pairsam_px2",
                    "cool",
                    "hic",
                    "mcool",
                    "normvector_juicerformat",
                    "zip",
                    "bg",
                    "bg_px2",
                    "bw",
                    "bed",
                    "bedpe",
                    "txt",
                    "csv",
                    "spt",
                    "other",
                    "bwaIndex",
                    "chromsizes",
                    "barcode_file",
                    "compressed_fasta",
                    "fasta",
                    "juicer_format_restriction_site_file",
                    "conc",
                    "merged_nodups"]


for wf in all_wf:
    #print(wf['display_title'])
    dt = wf['uuid']
    steps = wf.get('steps')
    #1 STEPS
    if not steps:
        print(dt,'NO STEPS')
    for step in steps:
        #2  Name, Inputs, Outputs, Meta
        for step_field in ['name', 'inputs', 'outputs', 'meta']:
            if step_field not in step.keys():
                print(dt,'NO {} IN STEP {}'.format(step_field, step['name']))
            elif not step[step_field]:
                print(dt,'NO VALUE FOR {} IN STEP {}'.format(step_field, step['name']))
#         #3 step meta should have software used
#         if step.get('meta'):
#             if 'software_used' not in step['meta'].keys():
#                 print(dt,'NO software_used in IN STEP {} meta'.format(step['name']))
        #4 name meta source in step inputs
        if step.get('inputs'):
            for an_input in step['inputs']:
                for input_field in ['meta', 'source', 'name']:
                    if input_field not in an_input.keys():
                        print(dt,'NO {} IN STEP INPUT {}'.format(input_field, step['name']))
                #5 input meta should have type file_format
                if an_input.get('meta'):
                    for input_meta_field in ['type']:
                        if input_meta_field not in an_input['meta'].keys():
                            print(dt,'NO {} IN STEP {} INPUT {} META'.format(input_meta_field, step['name'], an_input['name']))
                    if an_input['meta'].get('type'):
                        if an_input['meta']['type'] in ["data file", "reference file"]:
                            if 'file_format' not in an_input['meta'].keys():
                                print(dt,'NO file_format IN STEP {} INPUT {} META'.format(step['name'], an_input['name']))

                                # This part is not required anymore since file format is a linkTo
#                             else:
#                                 if an_input['meta']['file_format'] not in allowed_formats:
#                                     print(dt,'NO ALLOWED file_format {} IN STEP {} INPUT {} META'.format(an_input['meta']['file_format'], step['name'], an_input['name']))
                                    
                            
                #6 input source should have name
                if an_input.get('source'):
                    for a_source in an_input['source']:
                        for input_source_field in ['name']:
                            if input_source_field not in a_source.keys():
                                print(dt,'NO {} IN STEP INPUT SOURCE {}'.format(input_source_field, step['name']))
        #7 name target in step outputs
        if step.get('outputs'):
            for an_output in step['outputs']:
                for output_field in ['name', 'target']:
                    if output_field not in an_output.keys():
                        print(dt,'NO {} IN STEP OUTPUT {}'.format(output_field, step['name']))
                #8 target needs name
                if an_output.get('target'):
                    for a_target in an_output['target']:
                        if 'name' not in a_target.keys():
                            print(dt,'NO name IN STEP OUTPUT TARGET {}'.format(step['name']))
    if wf.get('arguments'):
        for arg in wf['arguments']:
            if arg.get('argument_format'):
                if arg['argument_format']['display_title'] not in allowed_formats:
                    print(dt,'NO ALLOWED type in argument format {}'.format(arg['argument_format']))
                    
                            
                    
                
# Required properties in “steps.outputs” objects:
# - Name
# - Target
# Required properties in “steps.outputs.target” objects:
# - Name           
# "properties": {
#                                         "type" : {
#                                             "title" : "Input Type",
#                                             "type" : "string",
#                                             "enum": [
#                                                 "data file",
#                                                 "report",
#                                                 "QC",
#                                                 "parameter",
#                                                 "reference file"
#                                             ]

# "file_format": "chromsizes",
# "file_format": "merged_nodups"

bef50397-4d72-4ed1-9c78-100e14e5c47f NO file_format IN STEP Provenance Tracking INPUT inputs META


Required properties in workflow schema:
- steps
Required properties in “steps” objects:
- Name
- Inputs
- Outputs
- Meta
Required properties in “steps.meta” objects:
- software_used
Required properties in “steps.inputs” objects:
- Name
- Meta
- Source
Required properties in “steps.inputs.meta” objects:
- Type
~- file_format~
Required properties in “steps.inputs.source” objects:
- name
Required properties in “steps.outputs” objects:
- Name
- Target
Required properties in “steps.outputs.target” objects:
- Name